In [1]:
import segmentation_models as sm
import numpy as np
import glob
import os
import cv2


# self-written scripts
import sys
sys.path.insert(0, 'Python_Scripts')

import data_preparation_cnn

Segmentation Models: using `keras` framework.


In [2]:
def get_images(class_id, size_x, size_y):
    images = []
    path_suffix = 'c' + str(class_id) + '/'

    for directory_path in glob.glob('data/segmentation/test/' + path_suffix):
        for img_path in sorted(glob.glob(os.path.join(directory_path, "*.jpg"))):
            #print(img_path)
            #break
            img = cv2.imread(img_path, cv2.IMREAD_COLOR)       
            img = cv2.resize(img, (size_y, size_x))
            img = img/255
            #print(img)
            #img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
            images.append(img)
            #train_labels.append(label)
    #Convert list to array for machine learning processing        
    images = np.array(images)
    
    return images

In [3]:
#get_images(1,128,128)

In [4]:
def get_masks(class_id, size_x, size_y):
    images = []
    path_suffix = 'c' + str(class_id) + '/'

    for directory_path in glob.glob('data/segmentation/test_mask/' + path_suffix):
        for img_path in sorted(glob.glob(os.path.join(directory_path, "*.jpg"))):
            #print(img_path)
            #break
            img = cv2.imread(img_path, 0)       
            img = cv2.resize(img, (size_y, size_x))
            #img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
            images.append(img)
            #train_labels.append(label)
    #Convert list to array for machine learning processing        
    images = np.array(images)
    
    return images

In [5]:
def prepare_input_variables(class_id, train_images, train_masks, size_x, size_y):
    # get preprocessing for `EfficientNetB5`
    preprocess_input = sm.get_preprocessing('efficientnetb5')
    
    # prepare variables
    x_train = preprocess_input(train_images)
    y_train = np.expand_dims(train_masks, axis=3)
    # y_train = y_train/255

    x_val = get_images(class_id=class_id, size_x=size_x, size_y=size_y)
    x_val = preprocess_input(x_val)
    y_val = get_masks(class_id=class_id, size_x=size_x, size_y=size_y)
    y_val = np.expand_dims(y_val, axis=3) #May not be necessary.. leftover from previous code 
    # y_val = y_val / 255

    return x_train, y_train, x_val, y_val

In [6]:
def build_compiled_model(size_x, size_y, metric_for_model):
    # set the correct framework for the model to work
    sm.set_framework('tf.keras')
    sm.framework()
    
    # define model
    model = sm.Unet('efficientnetb5',
                    #input_shape=(size_x, size_y, 3),
                    classes=1,
                    activation='sigmoid',
                    encoder_weights='imagenet',
                    encoder_freeze=True
                   )
    
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=[metric_for_model])
    
    return model

In [11]:
def get_history_from_mask_model(class_id, size_x, size_y, metric_for_model):
    # load images and masks as input for model
    train_images, train_masks = data_preparation_cnn.get_resized_image_and_mask_lists(class_id=class_id, 
                                                                              size_x=size_x, 
                                                                              size_y=size_y)

    # build input variables for model
    x_train, y_train, x_val, y_val = prepare_input_variables(class_id, 
                                                             train_images, 
                                                             train_masks, 
                                                             size_x, 
                                                             size_y)
        
    # create a compiled model
    model = build_compiled_model(size_x, size_y, metric_for_model)
    
    print(f'beginning training with masks')
    # fit the model
    history = model.fit(x_train, 
                        y_train,
                        batch_size=32, 
                        epochs=3,
                        verbose=True,
                        validation_data=(x_val, y_val)
                       )
    
    # save model
    model_name = 'models/class' + str(class_id) + '_mask_generator.h5'
    model.save(model_name)
    
    return history

In [12]:
def build_mask_generating_models(size_x, size_y, metric_for_model):
    
    losses = []
    losses_val = []
    
    metric_results = []
    metric_results_val = []
    
    for class_id in [1,2,3,4]:
        print(f'building model for defect class {class_id}...')
        
        # extract the history for each model
        history = get_history_from_mask_model(class_id, size_x, size_y, metric_for_model)
        
        losses.append(history.history['loss'])
        losses_val.append(history.history['val_loss'])
        
        metric_results = history.history[metric_for_model]
        metric_results_val = history.history['val_' + metric_for_model]
        
        
        
        print('model successfully generated and saved to file!')
        print('-----'*10)
        print()
        

In [13]:

# PARAMETERS
SIZE_X = 128
SIZE_Y = 128

In [14]:
build_mask_generating_models(SIZE_X, SIZE_Y, 'accuracy')

building model for defect class 1...
beginning training with masks
Epoch 1/3


2022-02-18 13:36:59.488194: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


32/32 [==============================] - ETA: 0s - loss: 0.2409 - accuracy: 0.9648

2022-02-18 13:39:41.189111: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


32/32 [==============================] - 182s 5s/step - loss: 0.2409 - accuracy: 0.9648 - val_loss: 0.3345 - val_accuracy: 0.9863
Epoch 2/3
32/32 [==============================] - 186s 6s/step - loss: 0.0720 - accuracy: 0.9898 - val_loss: 0.2027 - val_accuracy: 0.9887
Epoch 3/3
32/32 [==============================] - 246s 8s/step - loss: 0.0511 - accuracy: 0.9899 - val_loss: 0.0887 - val_accuracy: 0.9887


...

32/32 [==============================] - 392s 11s/step - loss: 0.2993 - accuracy: 0.9169 - val_loss: 2.3192 - val_accuracy: 0.0917
Epoch 2/3
32/32 [==============================] - 374s 12s/step - loss: 0.1749 - accuracy: 0.9438 - val_loss: 0.4077 - val_accuracy: 0.8349
Epoch 3/3
32/32 [==============================] - 341s 11s/step - loss: 0.1607 - accuracy: 0.9465 - val_loss: 0.6348 - val_accuracy: 0.7412
model successfully generated and saved to file!
--------------------------------------------------

building model for defect class 4...
beginning training with masks
Epoch 1/3


2022-02-18 14:20:13.138747: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


32/32 [==============================] - ETA: 0s - loss: 0.2813 - accuracy: 0.9166 

2022-02-18 14:26:08.107168: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


32/32 [==============================] - 381s 11s/step - loss: 0.2813 - accuracy: 0.9166 - val_loss: 4.7091 - val_accuracy: 0.5601
Epoch 2/3
32/32 [==============================] - 280s 9s/step - loss: 0.1467 - accuracy: 0.9484 - val_loss: 1.5033 - val_accuracy: 0.6028
Epoch 3/3
32/32 [==============================] - 259s 8s/step - loss: 0.1234 - accuracy: 0.9542 - val_loss: 0.3237 - val_accuracy: 0.9163
model successfully generated and saved to file!
--------------------------------------------------



In [ ]:
def dice_coef(y_true,y_pred):
    y_true_f=tf.reshape(tf.dtypes.cast(y_true, tf.float32),[-1])
    y_pred_f=tf.reshape(tf.dtypes.cast(y_pred, tf.float32),[-1])
    intersection=tf.reduce_sum(y_true_f*y_pred_f)
    return (2. * intersection + 1.) / (tf.reduce_sum(y_true_f) + tf.reduce_sum(y_pred_f) + 1.)
def dice_loss(y_true, y_pred):
    y_true_f = tf.reshape(y_true, [-1])
    y_pred_f = tf.reshape(y_pred, [-1])
    return (1-dice_coefficient(y_true, y_pred))
def bce_dice_loss(y_true, y_pred):
    y_true_f = tf.reshape(y_true, [-1])
    y_pred_f = tf.reshape(y_pred, [-1])
    return binary_crossentropy(y_true, y_pred) + (1-dice_coef(y_true, y_pred))